# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [35]:
# Top packages - stdlib, bottom packages - 3rd party
import re
import os
import glob
import time
import json
import csv
import traceback
import numpy as np
import pandas as pd
import cassandra

#### Creating list of filepaths to process original event csv data files

In [36]:
start_time = time.time()

# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [37]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
     # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

print(f"Row count: {len(full_data_rows_list)}")

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables - Old code
# csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)
# with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
#     writer = csv.writer(f, dialect='myDialect')
#     writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
#                 'level','location','sessionId','song','userId'])
#     for row in full_data_rows_list:
#         if (row[0] == ''):
#             continue
#         writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
        
full_rows_df = pd.DataFrame(([row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]] for row in full_data_rows_list), 
             columns = ['artist','firstName','gender','itemInSession','lastName','length','level','location','sessionId','song','userId'])

# Can't drop empty strings directly, so replace with nan before dropping
full_rows_df.replace('', np.nan, inplace=True)
full_rows_df.dropna(inplace=True)
full_rows_df.to_csv('event_datafile_df.csv', quoting=csv.QUOTE_ALL, encoding='utf8', index=False)

# Uncomment the code below to rename columns and create 3 separate files for running the CQLSH COPY command
full_rows_df.rename(columns={'firstName': 'first_name', 'itemInSession': 'item_in_session', 'length': 'song_length',
                             'lastName': 'last_name', 'sessionId': 'session_id', 'userId': 'user_id'}, inplace=True)

# Query 1 CSV
full_rows_df[['artist', 'song', 'song_length', 'session_id', 'item_in_session']].to_csv('event_datafile_songplay_session.csv', quoting=csv.QUOTE_ALL, encoding='utf8', index=False)

# Query 2 CSV
full_rows_df[['artist', 'song', 'first_name', 'last_name', 'user_id', 'item_in_session', 'session_id']].to_csv('event_datafile_songplay_user_session.csv', quoting=csv.QUOTE_ALL, encoding='utf8', index=False)

# Query 3 CSV
full_rows_df[['user_id', 'song', 'first_name', 'last_name']].to_csv('event_datafile_user_song_history.csv', quoting=csv.QUOTE_ALL, encoding='utf8', index=False)


Row count: 8056


In [38]:
# check the number of rows in your csv file - old csv file referred
# with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
#     print(sum(1 for line in f))

with open('event_datafile_df.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))


6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

### Getting spread statistics to optimize choice of clustering columns

Spread statistics (value_counts, mode, variance, standard deviation) provide an idea of how the data is distributed.
For text columns, we can compute mode and value_counts

In [39]:
df = pd.read_csv('event_datafile_new.csv')
headers = ['itemInSession', 'userId', 'sessionId', 'artist']

stats_dict = { header : [len(df[header].value_counts()), df[header].mode()[0]] for header in headers}
pd.DataFrame(stats_dict, index=['No of distinct values (value counts)', 'Most frequent occurrence (mode)'])

,itemInSession,userId,sessionId,artist
No of distinct values (value counts),123,96,776,3148
Most frequent occurrence (mode),1,49,589,Coldplay


## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [40]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [41]:
try:
    session.execute("""
                    CREATE KEYSPACE IF NOT EXISTS udacity 
                    WITH REPLICATION = 
                    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
                    """)

except Exception as e:
    print(e)

#### Set Keyspace

In [42]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [43]:
## Writing the queries below for easing data modeling process
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
## Query1 : "SELECT artist, song, length FROM songplay_session WHERE session_id = 338 AND item_in_session = 4"
## Query2 :  "SELECT artist, song,first_name, last_name AS full_name FROM songplay_user_session WHERE user_id = 10 AND session_id = 182"
## Query3 : "SELECT first_name, last_name FROM user_song_history WHERE song = 'All Hands Against His Own'"                  

### Create Table Queries

In [44]:
# Bundling create queries at the beginning for easier reading
create_query1 = """
                CREATE TABLE IF NOT EXISTS songplay_session (
                    artist text,
                    song text,
                    song_length float,
                    session_id int,
                    item_in_session int,
                    PRIMARY KEY (item_in_session, session_id)
                );
                """

create_query2 = """
                CREATE TABLE IF NOT EXISTS songplay_user_session (
                    artist text,
                    song text,
                    first_name text,
                    last_name text,
                    user_id int,
                    item_in_session int,
                    session_id int,
                    PRIMARY KEY ((user_id, session_id), item_in_session)
                ) 
                WITH CLUSTERING ORDER BY (item_in_session asc);
                """

create_query3 = """
                CREATE TABLE IF NOT EXISTS user_song_history (
                    user_id int,
                    song text,
                    first_name text,
                    last_name text,
                    PRIMARY KEY (song, first_name, last_name, user_id)
                );
                """
try:
    session.execute(create_query1);
    session.execute(create_query2);
    session.execute(create_query3);
except Exception:
    traceback.print_exc()

In [69]:
# Comment this entire cell out to load via cqlsh COPY command

# Read csv file and insert from data into table 1 for query 1
file = "event_datafile_df.csv"


insert_query1 = """
                    INSERT INTO songplay_session (artist, song, song_length, session_id, item_in_session)
                    VALUES (%s, %s, %s, %s, %s);
                """
insert_query2 = """
                    INSERT INTO songplay_user_session (artist, song, first_name, last_name, user_id, item_in_session, session_id)
                    VALUES (%s, %s, %s, %s, %s, %s, %s);
                """
insert_query3 = """
                    INSERT INTO user_song_history (user_id, song, first_name, last_name)
                    VALUES (%s, %s, %s, %s);
                """
print(f"Size of csv file: {os.stat(file).st_size/1024} KB")

# Chunking reads file in blocks to prevent memory load/possible crashing
try:
    # If reading a large csv, pass chunksize parameter and iterate through chunks
    # with for chunk in df_chunks:
    # If reading large CSV blobs from Amazon S3 or Azure Blob Storage, the byte streams would need to be chunked and decoded
    # in utf8 format, before splitting on newline character \n. This would create a partial chunk with half row and several complete rows

    df = pd.read_csv(file, encoding = "utf8")
    
    for index, row in df.iterrows():
        session.execute(insert_query1, (row["artist"], row["song"], row["length"], row["sessionId"], row["itemInSession"]))
        session.execute(insert_query2, (row["artist"], row["song"], row["firstName"], row["lastName"], 
                                            row["userId"], row["itemInSession"], row["sessionId"]))
        session.execute(insert_query3, (row["userId"], row["song"], row["firstName"], row["lastName"]))

except Exception:
        traceback.print_exc()

Size of csv file: 827.4326171875 KB


In [45]:
songplay_session_file_name = 'event_datafile_songplay_session.csv'
songplay_user_session_file_name = 'event_datafile_songplay_user_session.csv'
user_song_history_file_name = 'event_datafile_user_song_history.csv'

# COPY command to upload rows at high speed
!cqlsh -e "COPY udacity.songplay_session(artist, song, song_length, session_id, item_in_session) FROM '{songplay_session_file_name}' WITH header=TRUE" 

!cqlsh -e "COPY udacity.songplay_user_session(artist, song, first_name, last_name, user_id, item_in_session, session_id) FROM '{songplay_user_session_file_name}' WITH header=TRUE"

!cqlsh -e "COPY udacity.user_song_history(user_id, song, first_name, last_name) FROM '{user_song_history_file_name}' WITH header=TRUE"

Using 1 child processes

Starting copy of udacity.songplay_session with columns [artist, song, song_length, session_id, item_in_session].
Processed: 5000 rows; Rate:    3362 rows/s; Avg. rate:    3362 rows/sProcessed: 6820 rows; Rate:    3621 rows/s; Avg. rate:    3486 rows/sProcessed: 6820 rows; Rate:    1811 rows/s; Avg. rate:    3316 rows/s
6820 rows imported from 1 files in 2.057 seconds (0 skipped).
Using 1 child processes

Starting copy of udacity.songplay_user_session with columns [artist, song, first_name, last_name, user_id, item_in_session, session_id].
Processed: 5000 rows; Rate:    3080 rows/s; Avg. rate:    3080 rows/sProcessed: 6820 rows; Rate:    3245 rows/s; Avg. rate:    3162 rows/sProcessed: 6820 rows; Rate:    1623 rows/s; Avg. rate:    3021 rows/s
6820 rows imported from 1 files in 2.258 seconds (0 skipped).
Using 1 child processes

Starting copy of udacity.user_song_history with columns [user_id, song, first_name, last_name].
Processed: 5000 rows; Rate:    3058 row

### Query 1. artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [46]:
## SELECT statement to verify the data was entered into the table
select_query1 = """ SELECT artist, song, song_length
                    FROM songplay_session 
                    WHERE session_id = 338
                    AND item_in_session = 4;
                """
try:
    rows = session.execute(select_query1)
except Exception:
    traceback.print_exc()
    
for row in rows:
    print ((row.artist, row.song, row.song_length))

('Faithless', 'Music Matters (Mark Knight Dub)', 495.30731201171875)


### Query2: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [47]:
## SELECT statement to verify the data was entered into the table
select_query2 = """ SELECT artist, song, first_name, last_name
                    FROM songplay_user_session 
                    WHERE user_id = 10
                    AND session_id = 182
                    ORDER BY item_in_session;
                """
try:
    rows = session.execute(select_query2)
except Exception:
    traceback.print_exc()
    
for row in rows:
    print ((row.artist, row.song, row.first_name + " " + row.last_name))

('Down To The Bone', "Keep On Keepin' On", 'Sylvie Cruz')
('Three Drives', 'Greece 2000', 'Sylvie Cruz')
('Sebastien Tellier', 'Kilometer', 'Sylvie Cruz')
('Lonnie Gordon', 'Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', 'Sylvie Cruz')


### Query 3:every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [48]:
## SELECT statement to verify the data was entered into the table
select_query3 = """ SELECT first_name, last_name
                    FROM user_song_history 
                    WHERE song = 'All Hands Against His Own' ;
                """
try:
    rows = session.execute(select_query3)
except Exception:
    traceback.print_exc()
    
for row in rows:
    print (row.first_name + " " + row.last_name)


Jacqueline Lynch
Sara Johnson
Tegan Levine


In [74]:
end_time = time.time()
print(f"Time taken to read, insert and select queries: {end_time - start_time} s")

Time taken to read, insert and select queries: 25.321012020111084 s


### Drop the tables before closing out the sessions

In [49]:
drop_query1 = """ DROP TABLE IF EXISTS songplay_session;"""
drop_query2 = """ DROP TABLE IF EXISTS songplay_user_session;"""
drop_query3 = """ DROP TABLE IF EXISTS user_song_history;"""
session.execute(drop_query1);
session.execute(drop_query2);
session.execute(drop_query3);

### Close the session and cluster connection

In [50]:
session.shutdown()
cluster.shutdown()